# Market Research Systematic Quantitative Experimentation 2
In this notebook, we run market research several market research simulation experiments in order to compute quantitative metrics and compare Control to Treatment conditions.

In [ ]:
from pprint import pprint

from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe.utils.parallel import parallel_map_dict, parallel_map_cross

from tinytroupe.experimentation import InPlaceExperimentRunner

from tinytroupe.validation import persona_adherence, self_consistency, fluency, task_completion, divergence

# specific utilities
from common_utils import *
from market_research_utils import *

## Parameters

In [2]:
full_mode = False  # set to True to run the full mode with all agents and tasks

In [3]:
if full_mode:
    repetitions_per_task = 5
    simulation_steps = 3

else:
    repetitions_per_task = 1
    simulation_steps = 3
    qty_agents = 5
    qty_proposals = 10


## Auxiliary functions

In [4]:
def market_research_battery_b(agents, proposals, agent_propositions, environment_propositions, 
                            repetitions=1, simulation_steps=4): 
    
    agent_propositions_scores = {}
    environment_propositions_scores = {}

    print("Proposals:", proposals)

    experiments_count = 0
    total_expected_experiments = len(proposals) * repetitions #* len(agents)

    # loop over proposals
    for proposal in proposals:
        for i in range(repetitions):
            print("\n############## STARTING A NEW RESEARCH SESSION #################")
            print(f"Overall experiment number: {experiments_count+1} / {total_expected_experiments}")
            print(f"Proposal: {proposal}")
            print(f"Trial number: {i+1}")
            print(f"Customers: {agents}")

            world = TinyWorld(agents= agents, broadcast_if_no_target=False)

            # prepare customers 
            for customer in agents:
                # clear the episodic memory of all agents
                for person in world.agents:
                    person.clear_episodic_memory()
                    
                customer.listen(\
                    """
                    You are going to be interviewed for a market research about a new product or service.
                    Wait for the questions and answer them honestly. Please stay quiet until the you are asked something.
                    """
                    )
            
            # prepare the researcher
            interviewer_main_question =\
                f""" 
                We are developing a new product/service proposal, and would like to know if you would buy it or not.
                The proposal is the following:

                    "{proposal}"
                
                The question is:  would buy this product/service, if it were available in the market? If so, why? If not, why not?
                Feel free to carefully consider the proposal and let us know your thoughts about it. 

                Please consider all of your particularities, don't give just a general justifcation, but instead 
                dig deep into your own preferences,
                personality, style, behaviors, occupation, emotions, past history, etc. 
                We want a detailed and highly personalized justification.

                Please be honest, we are not here to judge you, but just to learn from you. 
                We know your preferences and choices depend on many factors, but please make your best guess.
                To do so, reflect deeply about your personality, interests, preferences, finances, emotions, etc., in order
                to provide a good answer. Take the time to think before talking.

                Now please answer the question.
                """

            # now to the discussions
            world.broadcast(interviewer_main_question)
            world.run(simulation_steps)
            
            world.broadcast("Please explain this better.")
            world.run(1)

            world.broadcast("Ok, at last, just to be clear, give a YES or NO answer. Make your best guess, even if not perfect.")
            world.run(1)
            
            experiments_count += 1

           # Evaluate environment propositions in parallel
            env_results = parallel_map_dict(
                environment_propositions,
                lambda item: item[1].score(
                    world, 
                    claim_variables={"task_description": f"A market research session was run about: {proposal}."}, 
                    return_full_response=True
                )
            )
            
            # Process environment results
            for k, result in env_results.items():
                if k not in environment_propositions_scores:
                    environment_propositions_scores[k] = []
                environment_propositions_scores[k].append(result["value"])
                print(result)

            # Evaluate agent propositions across all agents in parallel
            agent_results = parallel_map_cross(
                [agents, agent_propositions.items()],
                lambda agent, prop_item: (
                    prop_item[0],  # proposition key
                    prop_item[1].score(agent, return_full_response=True)  # result
                )
            )
            
            # Process agent results
            for k, result in agent_results:
                if k not in agent_propositions_scores:
                    agent_propositions_scores[k] = []
                agent_propositions_scores[k].append(result["value"])
                print(result)

    return agent_propositions_scores, environment_propositions_scores

## Experiment setup

In [ ]:
experiment_runner = InPlaceExperimentRunner("./market_research_quantitative_experimentation_2.json")

experiment_runner.add_experiment("Control")
experiment_runner.add_experiment("Treatment")

In [6]:
experiment_runner.activate_next_experiment()

#experiment_runner.fix_active_experiment("Control")
#experiment_runner.fix_active_experiment("Treatment")

In [7]:
print(f"Running experiment {experiment_runner.get_active_experiment()}")

Running experiment Control


## Agents and populations

In [8]:

# avoid displaying the communication, to make the output cleaner for eval
TinyPerson.communication_display = True

people = []
if not experiment_runner.has_finished_all_experiments():
    # load agents
    people = TinyPerson.load_specifications_from_folder("./population/global_general")

    # filter to make it go faster?
    if not full_mode:
        people = people[:qty_agents]

    # customize and print minibios 
    for person in people:
        ##person.import_fragment("./fragments/picky_customer.agent.fragment.json")
        print(person.minibio())


Amelia Scott is a 30 year old Operations Manager, British, currently living in London, England. Amelia Scott is not only a dedicated Operations Manager but also a passionate advocate for continuous learning and diversity in the workplace. Her strong leadership skills are complemented by her empathetic nature, allowing her to connect with team members and foster a collaborative environment. Outside of work, Amelia enjoys cooking elaborate meals and experimenting with new recipes, often hosting dinner parties for friends and family. She is also an avid traveler, having explored over 15 countries, and is committed to giving back to her community through volunteering and promoting mental health awareness.
Ava Patel is a 18 year old Student, Brazilian, currently living in São Paulo, Brazil. Ava Patel is not only a dedicated student but also a compassionate individual who deeply cares about social justice and environmental issues. With a strong sense of empathy, he often prioritizes the need

In [9]:
len(people)

5

In [10]:
# divide people in several groups of 5
people_groups = []
for i in range(0, len(people), 5):
    people_groups.append(people[i:i+5]
    )

len(people_groups)

1

In [11]:
# The experiment refers to customers

if experiment_runner.get_active_experiment() == "Control":
    for person in people:
        person.action_generator.enable_reasoning_step = False
        person.action_generator.enable_quality_checks = False

elif experiment_runner.get_active_experiment() == "Treatment":    
    for person in people:
       person.action_generator.enable_reasoning_step = False
       person.action_generator.enable_quality_checks = True
       person.action_generator.max_attempts = 3
       person.action_generator.enable_regeneration = True
       person.action_generator.quality_threshold = 8  

## Product and service proposals

In [12]:

proposals = [
    "Bottled gazpacho to be sold in supermarkets outside of Spain",
    "Self-cleaning gym equipment handles to enhance hygiene",
    "Drone delivery service specialized in sending freshly baked bread to remote locations",
    "An app for luxury travel, particularly for places where kids are not allowed",
    "A subscription to discount coupons for supermarkets, ensuring always the lowest prices, even if at the cost of quality",
    "A subscription to a service that sends you a new, expensive, luxury item every month, without you having to choose it.",
    "A wearable device that alerts you before you forget personal items at cafes or restaurants",
    "Instant-drying, reusable notebooks that digitally sync handwritten notes automatically",
    "Subscription-based personalized vitamins delivered weekly based on real-time health metrics",
    "A smart coffee mug that maintains precise drink temperatures and tracks hydration",
    "An app that uses AI to instantly summarize lengthy documents into actionable tasks",
    "Biodegradable disposable utensils infused with plant seeds for gardening purposes",
    "Wireless earbuds that automatically translate foreign languages in real-time",
    "A subscription box featuring unique snacks from fictional universes (books, movies, games)",
    "Anti-anxiety bracelets using gentle pulse technology to help manage stress levels",
    "Smart fridge shelves that monitor expiration dates and order groceries automatically",
    "An AI-driven wardrobe organizer suggesting daily outfits based on weather and activities",
    "Carbon-negative sneakers made entirely from ocean waste materials",
    "Voice-activated, self-rotating plant pots that ensure optimal sunlight exposure",
    "Virtual reality meditation experiences tailored to specific emotional states",
    "A portable device that instantly cools or heats drinks on demand",
    "Subscription service delivering weekly eco-friendly cleaning supplies based on household needs",
    "Sleep-enhancing pillow equipped with temperature regulation and sleep analytics",
    "App-enabled plant sensors that alert users precisely when and how to care for their plants",
    "A handheld scanner identifying food allergens instantly from dishes",
    "AI-powered skin patch that detects and monitors hydration levels throughout the day",
    "Smart bike helmets featuring integrated safety lights, signals, and accident detection",
    "Interactive, holographic fitness instructors for home workouts",
    "Subscription-based personalized artwork curated by AI to match your home decor style",
    "Portable solar chargers designed specifically for charging electric bikes and scooters",
    "Self-driving mobile offices, turning commute time into productive working hours",
    "Edible, nutrient-packed water pods that replace disposable bottles during sports events",
    "Wireless charging mats embedded in clothing to charge devices discreetly",
    "Subscription service for exclusive mystery puzzle experiences mailed monthly",
    "Wearable mood trackers that recommend tailored playlists and podcasts in real-time",
    "Home robots specialized in folding laundry and organizing wardrobes autonomously",
    "Digital mirrors allowing users to virtually preview makeup or hairstyles in real-time",
    "Subscription boxes providing ingredients for authentic meals from extinct historical cuisines",
    "AI-powered personal finance app that predicts spending habits and suggests budget adjustments",
    "Solar-powered tents equipped with climate control and Wi-Fi for off-grid camping",
    "Miniature drone cameras for personal photography that autonomously follow users",
    "Subscription-based kits to build personalized, functioning miniature ecosystems at home",
    "Instant voice-to-braille devices allowing real-time conversation accessibility",
    "Eco-friendly smart tags for luggage that track location and reduce airline baggage loss"
]

if not full_mode:
    proposals = proposals[:qty_proposals]


## Perform the research



In [13]:
agent_propositions_scores={}
environment_propositions_scores={}

In [14]:
def research(people):
    global agent_propositions_scores, environment_propositions_scores
    if not experiment_runner.has_finished_all_experiments():
        tmp_agent_propositions_scores, tmp_environment_propositions_scores = \
            market_research_battery_b(
                agents=people,
                proposals=proposals,

                agent_propositions={
                    "Persona Adherence": persona_adherence,
                    "Self-consistency": self_consistency,
                    "Fluency": fluency
                },
                environment_propositions={
                    #"Task Completion": task_completion_proposition,
                    #"Divergence": divergence_proposition
                },
                repetitions=repetitions_per_task,
                simulation_steps=simulation_steps
            )

        pprint("NEW AGENT PROPOSITIONS SCORES")
        pprint(tmp_agent_propositions_scores)
        print("\n\n")
        pprint("NEW ENVIRONMENT PROPOSITIONS SCORES")
        pprint(tmp_environment_propositions_scores)

        # merge the scores lists
        agent_propositions_scores = merge_dicts_of_lists(tmp_agent_propositions_scores, agent_propositions_scores)
        environment_propositions_scores = merge_dicts_of_lists(tmp_environment_propositions_scores, environment_propositions_scores)

        return agent_propositions_scores, environment_propositions_scores

To make it easier to visualize the outputs, we'll split the experiment in several groups. This ensures the simulation outputs are visible in a single cell output.

In [15]:
research(people_groups[0]) if len(people_groups) > 0 else None

Proposals: ['Bottled gazpacho to be sold in supermarkets outside of Spain', 'Self-cleaning gym equipment handles to enhance hygiene', 'Drone delivery service specialized in sending freshly baked bread to remote locations', 'An app for luxury travel, particularly for places where kids are not allowed', 'A subscription to discount coupons for supermarkets, ensuring always the lowest prices, even if at the cost of quality', 'A subscription to a service that sends you a new, expensive, luxury item every month, without you having to choose it.', 'A wearable device that alerts you before you forget personal items at cafes or restaurants', 'Instant-drying, reusable notebooks that digitally sync handwritten notes automatically', 'Subscription-based personalized vitamins delivered weekly based on real-time health metrics', 'A smart coffee mug that maintains precise drink temperatures and tracks hydration']

############## STARTING A NEW RESEARCH SESSION #################
Overall experiment numb

USER --> Amelia Scott: [CONVERSATION] 
          >                      You are going to be interviewed for a market research about a new
          > product or service.                     Wait for the questions and answer them honestly.
          > Please stay quiet until the you are asked something.

       + --> Ava Patel

       + --> Benjamin Taylor

       + --> ...others...

...

Charlotte Nguyen acts: [DONE] 

Ava Patel acts: [DONE] 

Amelia Scott acts: [DONE] 

{'reasoning': "To evaluate the proposition regarding Amelia Scott's adherence to her persona specification, I will analyze her actions in the simulation trajectory against the criteria outlined in her persona. The criteria include personality traits, persona style, beliefs, behaviors, skills, and any other aspects of the persona specification. \n\n1. **Personality Traits**: Amelia's actions reflect her ambitious and driven nature, as she engages thoughtfully with the product proposal and considers its implications for her efficiency and health. This aligns with her persona traits. \n\n2. **Persona Style**: Her communication style is professional and assertive, which is evident in her detailed responses and structured thinking about the product. This is consistent with her persona specification. \n\n3. **Beliefs**: Amelia's consideration of the product's practicality and cost reflects her belief in continuous learning and the importance of fulfilling work. She evaluates the product base

({'Persona Adherence': [9,
   8,
   9,
   8,
   8,
   9,
   9,
   9,
   9,
   8,
   8,
   8,
   9,
   9,
   9,
   9,
   9,
   9,
   8,
   8,
   8,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   8,
   8,
   9,
   8,
   9,
   9,
   9,
   9,
   9,
   8,
   8,
   9,
   9,
   8,
   9,
   9,
   8,
   9,
   8,
   9],
  'Self-consistency': [9,
   8,
   7,
   9,
   3,
   9,
   8,
   8,
   8,
   9,
   8,
   9,
   8,
   9,
   9,
   4,
   9,
   8,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   9,
   8,
   8,
   9,
   9,
   9,
   9,
   9,
   8,
   4,
   3,
   8,
   9,
   7,
   9,
   8,
   9,
   9,
   9],
  'Fluency': [8,
   7,
   7,
   8,
   7,
   8,
   7,
   9,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   8,
   7,
   7,
   7,
   9,
   8,
   8,
   8,
   8,
   7,
   7,
   8,
   7,
   8,
   8,
   8,
   8,
   9,
   7,
   8,
   8,
   8,
   8,
   7,
   7,
   8,
   8,
   8,
   8,
   8,
   8,
   7]},
 {})

In [16]:
research(people_groups[1]) if len(people_groups) > 1 else None

In [17]:
research(people_groups[2]) if len(people_groups) > 2 else None

In [18]:
research(people_groups[3]) if len(people_groups) > 3 else None

In [19]:
research(people_groups[4]) if len(people_groups) > 4 else None

## Extract results and analyze

Now we can actually extract the results.

In [20]:
if experiment_runner.get_active_experiment() in ["Control", "Treatment"]:
    combined_scores = {**agent_propositions_scores, **environment_propositions_scores}
    experiment_runner.add_experiment_results(combined_scores, experiment_name=experiment_runner.get_active_experiment()) 
    
    plot_scores(combined_scores)

else:
    print("Experiment finished. No more experiments to run.")

{'Fluency': [8,
             7,
             7,
             8,
             7,
             8,
             7,
             9,
             8,
             8,
             8,
             8,
             8,
             8,
             8,
             8,
             8,
             8,
             8,
             7,
             7,
             7,
             9,
             8,
             8,
             8,
             8,
             7,
             7,
             8,
             7,
             8,
             8,
             8,
             8,
             9,
             7,
             8,
             8,
             8,
             8,
             7,
             7,
             8,
             8,
             8,
             8,
             8,
             8,
             7],
 'Persona Adherence': [9,
                       8,
                       9,
                       8,
                       8,
                       9,
                       9,
                 

,Proposition,Average Score,Standard Deviation,Count
0,Persona Adherence,8.66,0.478518,50.0
1,Self-consistency,8.24,1.519398,50.0
2,Fluency,7.78,0.545482,50.0


In [21]:
if experiment_runner.has_finished_all_experiments():
    print("All experiments have been finished.")
    print(f"STATISTICTS: Control vs")
    pprint(experiment_runner.run_statistical_tests(control_experiment_name='Control'))

    # plot scores of both experiments
    experiment_control_scores = experiment_runner.get_experiment_results("Control")
    experiment_treatment_scores = experiment_runner.get_experiment_results("Treatment")
    
    
    plot_scores(experiment_control_scores)
    plot_scores(experiment_treatment_scores)

else:
    print("Not all experiments have been finished. RESTART AND RERUN.")

Not all experiments have been finished. RESTART AND RERUN.
